<a href="https://colab.research.google.com/github/wojiushigexiaobai/Myblog/blob/main/docs/docs/tutorials/llm_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a simple LLM application with chat models and prompt templates
# 使用聊天模型和提示模板构建一个简单的LLM应用程序
在本快速入门中，我们将向您展示如何使用 LangChain 构建一个简单的LLM应用程序。此应用程序会将文本从英语翻译成另一种语言。这是一个相对简单的LLM应用程序 - 它只是一个LLM调用加上一些提示。尽管如此，这是开始使用 LangChain 的好方法 - 只需一些提示和调用LLM就可以构建很多功能！

阅读本教程后，您将对以下内容有一个大致的了解：

- 使用 [语言模型](/docs/concepts/chat_models)

- 使用 [提示模板](/docs/concepts/prompt_templates)

- 使用[LangSmith]调试和跟踪应用程序 (https://docs.smith.langchain.com/)

让我们开始吧！

## 设置

### Jupyter Notebook

This and other tutorials are perhaps most conveniently run in a [Jupyter notebooks](https://jupyter.org/). Going through guides in an interactive environment is a great way to better understand them. See [here](https://jupyter.org/install) for instructions on how to install.

本教程和其他教程可能在 Jupyter 笔记本中运行最方便。在交互式环境中浏览指南是更好地了解它们的好方法。有关如何安装的说明，请参阅此处。

### Installation 安装

要安装 LangChain，请运行：

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>



有关更多详细信息，请参阅我们的[安装指南](/docs/how_to/installation).

### LangSmith

您使用 LangChain 构建的许多应用程序将包含多个步骤和多次调用LLM。随着这些应用程序变得越来越复杂，能够检查您的链条或代理内部到底发生了什么变得至关重要。最好的方法是使用[LangSmith](https://smith.langchain.com).

在上面的链接中注册后，请确保设置环境变量以开始记录跟踪：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

或者，如果在笔记本中，您可以通过以下方式设置它们：

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

In [ ]:
!pip install -U langchain langchain-openai

In [3]:
import getpass
import os
from google.colab import userdata

os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')

In [ ]:
LANGSMITH_TRACING=true
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="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"
LANGSMITH_PROJECT="pr-terrible-halibut-84"
OPENAI_API_KEY="<your-openai-api-key>"

## Using Language Models 使用语言模型

首先，让我们学习如何单独使用语言模型。LangChain 支持许多不同的语言模型，您可以互换使用。有关开始使用特定模型的详细信息，请参阅[支持的集成](/docs/integrations/chat/).

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4o-mini"}}} />


In [15]:
# !pip install -qU "langchain[openai]"
!pip install -qU "langchain[groq]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 2.8 MB/s eta 0:00:00


In [ ]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [16]:
import getpass
import os
from google.colab import userdata

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = userdata.get('groq')

from langchain.chat_models import init_chat_model

model = init_chat_model("llama3-8b-8192", model_provider="groq")

Let's first use the model directly. [ChatModels](/docs/concepts/chat_models) are instances of LangChain [Runnables](/docs/concepts/runnables/), which means they expose a standard interface for interacting with them. To simply call the model, we can pass in a list of [messages](/docs/concepts/messages/) to the `.invoke` method.

我们首先直接使用模型。ChatModel 是 LangChain Runnables 的实例，这意味着它们公开了用于与它们交互的标准接口。要简单地调用模型，我们可以将消息列表传递给 .invoke 方法。

In [17]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 24, 'total_tokens': 28, 'completion_time': 0.003333333, 'prompt_time': 0.003384447, 'queue_time': 0.019616012000000002, 'total_time': 0.00671778}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-bf8e93d1-e787-4f2e-be43-6ad1e11584f8-0', usage_metadata={'input_tokens': 24, 'output_tokens': 4, 'total_tokens': 28})

:::tip

If we've enabled LangSmith, we can see that this run is logged to LangSmith, and can see the [LangSmith trace](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r). The LangSmith trace reports [token](/docs/concepts/tokens/) usage information, latency, [standard model parameters](/docs/concepts/chat_models/#standard-parameters) (such as temperature), and other information.

如果我们启用了 LangSmith，我们可以看到此运行已记录到 LangSmith 中，并且可以看到 LangSmith 跟踪。LangSmith 跟踪报告令牌使用信息、延迟、标准模型参数（例如温度）和其他信息。
:::

Note that ChatModels receive [message](/docs/concepts/messages/) objects as input and generate message objects as output. In addition to text content, message objects convey conversational [roles](/docs/concepts/messages/#role) and hold important data, such as [tool calls](/docs/concepts/tool_calling/) and token usage counts.

请注意，ChatModels 接收消息对象作为输入，并生成消息对象作为输出。除了文本内容之外，消息对象还传达会话角色并保存重要数据，例如工具调用和令牌使用计数。

LangChain also supports chat model inputs via strings or [OpenAI format](/docs/concepts/messages/#openai-format). The following are equivalent:

LangChain 还支持通过字符串或 OpenAI 格式输入聊天模型。以下是等效的：
```python
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])
```

### Streaming 流

Because chat models are [Runnables](/docs/concepts/runnables/), they expose a standard interface that includes async and streaming modes of invocation. This allows us to stream individual tokens from a chat model:

由于聊天模型是 Runnables，因此它们公开了一个标准接口，其中包括异步和流式调用模式。这允许我们从聊天模型中流式传输单个令牌：

In [18]:
for token in model.stream(messages):
    print(token.content, end="|")

|C|iao|!||

您可以在[本指南](/docs/how_to/chat_streaming/)中找到有关流式聊天模型输出的更多详细信息。

## Prompt Templates 提示模板

现在，我们将消息列表直接传递到语言模型中。此消息列表来自何处？通常，它是由用户输入和应用程序逻辑的组合构建的。此应用程序逻辑通常采用原始用户输入，并将其转换为准备传递给语言模型的消息列表。常见转换包括添加系统消息或使用用户输入设置模板格式。

[提示模板](/docs/concepts/prompt_templates/)是 LangChain 中的一个概念，旨在协助进行这种转换。它们接收原始用户输入并返回准备传递到语言模型的数据（提示）。

让我们在这里创建一个提示模板。它将接受两个用户变量：

- `language`: 要将文本翻译成的语言
- `text`:  要翻译的文本

In [19]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

Note that `ChatPromptTemplate` supports multiple [message roles](/docs/concepts/messages/#role) in a single template. We format the `language` parameter into the system message, and the user `text` into a user message.

请注意，ChatPromptTemplate 支持单个模板中的多个消息角色。我们将 language 参数格式化为系统消息，将用户文本格式化为用户消息。

此提示模板的输入是字典。我们可以单独使用这个提示模板，看看它自己做了什么

In [20]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

我们可以看到，它返回一个由两条消息组成的`ChatPromptValue`。如果我们想直接访问消息，我们会这样做：

In [21]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

最后，我们可以在格式化的提示符上调用 chat 模型：

In [22]:
response = model.invoke(prompt)
print(response.content)

Ciao!


:::tip
Message `content` can contain both text and [content blocks](/docs/concepts/messages/#aimessage) with additional structure. See [this guide](/docs/how_to/output_parser_string/) for more information.

消息内容可以包含具有附加结构的文本块和内容块。有关更多信息，请参阅本指南。
:::

If we take a look at the [LangSmith trace](https://smith.langchain.com/public/3ccc2d5e-2869-467b-95d6-33a577df99a2/r), we can see exactly what prompt the chat model receives, along with [token](/docs/concepts/tokens/) usage information, latency, [standard model parameters](/docs/concepts/chat_models/#standard-parameters) (such as temperature), and other information.

如果我们看一下 LangSmith 跟踪，我们可以准确地看到聊天模型收到了什么提示，以及令牌使用信息、延迟、标准模型参数（如温度）和其他信息。

## Conclusion 结论

就是这样！在本教程中，您学习了如何创建您的第一个简单LLM应用程序。您已经学习了如何使用语言模型、如何创建提示模板，以及如何在使用 LangSmith 创建的应用程序中实现出色的可观测性。

这只是触及了您想要学习成为熟练 AI 工程师的皮毛。幸运的是 - 我们还有很多其他资源！

For 要进一步了解 LangChain 的核心概念，我们提供了详细的[概念指南](/docs/concepts).

如果您对这些概念有更具体的问题，请查看作指南的以下部分：

- [聊天模型](/docs/how_to/#chat-models)
- [提示模板](/docs/how_to/#prompt-templates)

And the LangSmith docs:

- [LangSmith](https://docs.smith.langchain.com)